**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°: 1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.



In [35]:
# Pre procesamiento y web scrapping
import requests
from bs4 import BeautifulSoup
import pandas as pd

# modelado
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
import spacy;

### Lectura de DF

Descargamos los dataset desde el link, para no lidiar con inconvenientes al abrir y cerrar sesiones de colab.

In [2]:
!wget -O bgg_database.csv 'https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq'
!wget -O IMDB-Movie-Data.csv 'https://drive.google.com/uc?id=1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c'

--2024-11-01 13:27:50--  https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
Resolving drive.google.com (drive.google.com)... 74.125.141.138, 74.125.141.101, 74.125.141.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.141.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq [following]
--2024-11-01 13:27:50--  https://drive.usercontent.google.com/download?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.216.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1833113 (1.7M) [application/octet-stream]
Saving to: ‘bgg_database.csv’

bgg_database.csv    100%[===================>]   1.75M  --.-KB/s    in 0.01s   

2024-11-01 13:27:54 (141 MB

In [3]:
df_juegos: pd.DataFrame = pd.read_csv('bgg_database.csv')
df_peliculas: pd.DataFrame = pd.read_csv('IMDB-Movie-Data.csv')

### Exploración y procesado de df_juegos

Tomar criterios de evaluación para ver si borrar columnas o no.

In [4]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              1000 non-null   int64  
 1   game_name         1000 non-null   object 
 2   game_href         1000 non-null   object 
 3   geek_rating       1000 non-null   float64
 4   avg_rating        1000 non-null   float64
 5   num_voters        1000 non-null   float64
 6   description       1000 non-null   object 
 7   yearpublished     1000 non-null   int64  
 8   minplayers        1000 non-null   int64  
 9   maxplayers        1000 non-null   int64  
 10  minplaytime       1000 non-null   int64  
 11  maxplaytime       1000 non-null   int64  
 12  minage            1000 non-null   int64  
 13  avgweight         1000 non-null   float64
 14  best_num_players  1000 non-null   object 
 15  designers         1000 non-null   object 
 16  mechanics         1000 non-null   object 
 

Podemos visualizar que las siguientes columnas presentan datos que neceistamos trabajar:
- best_num_players: presenta una lista que contiene un diccionario.
- designers, mechanics, categories: son listas.
- description: tiene caracteres '\n'

In [5]:
df_juegos.sample(5)

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories
683,684,Zombicide,https://boardgamegeek.com/boardgame/113924/zom...,6.821,7.10,18923.0,Zombicide is a collaborative game in which pla...,2012,1,6,60,60,14,2.5336,"[{'min': 3, 'max': 3}, {'min': 6, 'max': 6}]","['Raphaël Guiton', 'Jean-Baptiste Lullien', 'N...","['Action Points', 'Cooperative Game', 'Dice Ro...","['Fighting', 'Horror', 'Miniatures', 'Zombies']"
109,110,Raiders of the North Sea,https://boardgamegeek.com/boardgame/170042/rai...,7.521,7.74,24324.0,Raiders of the North Sea is set in the central...,2015,2,4,60,80,12,2.5583,"[{'min': 3, 'max': 3}]",['Shem Phillips'],"['Contracts', 'Dice Rolling', 'Hand Management...","['Medieval', 'Nautical']"
997,998,Condottiere,https://boardgamegeek.com/boardgame/112/condot...,6.616,6.88,10260.0,It is 13th century Italy. Trade is flourishing...,1995,2,6,20,45,10,1.9591,"[{'min': 4, 'max': 5}]","['Dominique Ehrhard', 'Duccio Vitale', 'Justin...","['Area Majority / Influence', 'Auction/Bidding...","['Bluffing', 'Card Game', 'Medieval', 'Negotia..."
628,629,Lacrimosa,https://boardgamegeek.com/boardgame/348450/lac...,6.863,7.69,3686.0,Wolfgang Amadeus Mozart is dead. His last cons...,2022,1,4,90,90,14,3.1547,"[{'min': 3, 'max': 3}]","['Gerard Ascensi', 'Ferran Renalias']","['Area Majority / Influence', 'Deck, Bag, and ...",['Music']
122,123,Twilight Imperium: Third Edition,https://boardgamegeek.com/boardgame/12493/twil...,7.495,7.79,19304.0,Twilight Imperium Third Edition is an epic emp...,2005,3,6,180,240,14,4.2584,"[{'min': 6, 'max': 6}]",['Christian T. Petersen'],"['Action Points', 'Area-Impulse', 'Dice Rollin...","['Civilization', 'Negotiation', 'Political', '..."


In [6]:
df_juegos.describe()

,rank,geek_rating,avg_rating,num_voters,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,500.500000,7.053187,7.570630,13615.758000,2008.427000,1.760000,4.937000,59.55700,93.397000,11.636000,2.680625
std,288.819436,0.351866,0.391122,15876.211424,135.395031,0.689112,6.272509,43.50627,83.159816,2.252683,0.819375
min,1.000000,6.615000,6.760000,1364.000000,-2200.000000,1.000000,1.000000,0.00000,0.000000,0.000000,1.000000
25%,250.750000,6.770000,7.277500,4608.250000,2011.000000,1.000000,4.000000,30.00000,45.000000,10.000000,2.100300
50%,500.500000,6.970000,7.530000,7983.500000,2016.000000,2.000000,4.000000,60.00000,77.500000,12.000000,2.642300
75%,750.250000,7.256250,7.802500,16097.750000,2019.000000,2.000000,5.000000,60.00000,120.000000,14.000000,3.266025
max,1000.000000,8.415000,9.040000,128735.000000,2024.000000,6.000000,100.000000,480.00000,1200.000000,18.000000,4.818200


In [7]:
df_juegos.isna().sum() #no encontramos nulos.

,0
rank,0
game_name,0
game_href,0
geek_rating,0
avg_rating,0
num_voters,0
description,0
yearpublished,0
minplayers,0
maxplayers,0


### Exploración y procesado de df_peliculas

In [8]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Genre               1000 non-null   object 
 3   Description         1000 non-null   object 
 4   Director            1000 non-null   object 
 5   Actors              1000 non-null   object 
 6   Year                1000 non-null   int64  
 7   Runtime (Minutes)   1000 non-null   int64  
 8   Rating              1000 non-null   float64
 9   Votes               1000 non-null   int64  
 10  Revenue (Millions)  1000 non-null   float64
 11  Metascore           1000 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 93.9+ KB


In [9]:
df_peliculas.sample(5)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
12,13,Rogue One,"Action,Adventure,Sci-Fi",The Rebel Alliance makes a risky move to steal...,Gareth Edwards,"Felicity Jones, Diego Luna, Alan Tudyk, Donnie...",2016,133,7.9,323118,532.17,65
249,250,The Intouchables,"Biography,Comedy,Drama",After he becomes a quadriplegic from a paragli...,Olivier Nakache,"François Cluzet, Omar Sy, Anne Le Ny, Audrey F...",2011,112,8.6,557965,13.18,57
313,314,The Hobbit: The Battle of the Five Armies,"Adventure,Fantasy",Bilbo and Company are forced to engage in a wa...,Peter Jackson,"Ian McKellen, Martin Freeman, Richard Armitage...",2014,144,7.4,385598,255.11,59
34,35,Resident Evil: The Final Chapter,"Action,Horror,Sci-Fi",Alice returns to where the nightmare began: Th...,Paul W.S. Anderson,"Milla Jovovich, Iain Glen, Ali Larter, Shawn R...",2016,107,5.6,46165,26.84,49
828,829,Grindhouse,"Action,Horror,Thriller",Quentin Tarantino and Robert Rodriguez's homag...,Robert Rodriguez,"Kurt Russell, Rose McGowan, Danny Trejo, Zoë Bell",2007,191,7.6,160350,25.03,77


In [10]:
df_peliculas.isna().sum() # no encontramos nulos.

,0
Rank,0
Title,0
Genre,0
Description,0
Director,0
Actors,0
Year,0
Runtime (Minutes),0
Rating,0
Votes,0



## Web scrapping y procesado: df_libros


In [11]:
url: str = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'

In [12]:
response: requests.models = requests.get(url)

# extraigo el contenido de la página web
soup: BeautifulSoup = BeautifulSoup(response.text, 'html.parser')

# luego de la inspección, vemos que necesitamos la información de page_content
libro_div = soup.find('div', class_='page_content')

# divido para quedarme con los títulos
paragraphs = libro_div.find_all('a')

In [13]:
# creo lista para poder crear el df
libro: list = [par.text for par in paragraphs]
libro = libro[1::] # mediante slicing salteo la primera linea con información que no nos interesa.

In [14]:
df_libros: pd.DataFrame = pd.DataFrame(libro, columns=['Libro'])

In [15]:
def limpieza_top(libro: str) -> bool:
  """
  Chequea si en string existe el substring 'Top 1000'. Retorna un booleano
  """
  if 'Top 1000' in libro:
    return True
  else:
    return False

In [16]:
df_libros['es_ruido'] = df_libros['Libro'].apply(limpieza_top)

In [17]:
df_libros.drop(df_libros[df_libros['es_ruido'] == True].index, inplace=True)

In [18]:
df_libros = df_libros.drop('es_ruido', axis=1) # dropeo columna auxiliar.

In [19]:
df_libros.reset_index(drop=True, inplace=True) # reseteo indices, ya que eliminé las filas que no son libros.

In [20]:
df_libros = df_libros.drop_duplicates() # elimino duplicados si los hubiera

In [21]:
df_libros_filtered: pd.DataFrame = df_libros.copy()
df_libros_filtered['Libro'] = df_libros['Libro'].str.replace(r'\s*\(\d+\)$', '', regex=True)


### df_libros: segundo enfoque

Hacer scrapping sobre la pagina para obterner el link de referencia de cada libro.

Por ejermplo: 'Romeo and Juliet by William Shakespeare (2410)' hace referencia a '/ebook/1513'. De esa forma se puede ingresar a https://www.gutenberg.org/ebooks/1513 y obtener así una breve descripcion del libro (necesaria para la recomendacion final)


In [22]:
url: str = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'

In [23]:
"""
Esta celda se encarda de hacer web scrapping sobre la lista con todos los libros.
De ella, se obtiene tanto el título del libro como su link link de referencia.

La información es guardada sobre el dataframe 'books_df'

"""

response: requests.models = requests.get(url)
boup: BeautifulSoup = BeautifulSoup(response.text, 'html.parser')
libro_div = soup.find('div', class_='page_content')

books: list[dict[str,str]] = []

for a in libro_div.find_all('a', href=True):
    title = a.get_text(strip=True)
    link = a['href']
    books.append({'title': title, 'link': link})

books_df: pd.DataFrame = pd.DataFrame(books, columns=['title', 'link'])

In [24]:
# Se elimina todo elemento fuera del Top 1000
books_df = books_df.iloc[7:1007].reset_index(drop=True)

In [25]:
books_df.head()

,title,link
0,"Frankenstein; Or, The Modern Prometheus by Mar...",/ebooks/84
1,The Legend of Sleepy Hollow by Washington Irvi...,/ebooks/41
2,Pride and Prejudice by Jane Austen (2536),/ebooks/1342
3,"Moby Dick; Or, The Whale by Herman Melville (2...",/ebooks/2701
4,Romeo and Juliet by William Shakespeare (2132),/ebooks/1513


Una vez obtenido el DataFrame con los libros pertenecientes al Top 1000, nos resulta de interés tener una breve descripción sobre cada libro.  

Para ello, se itera sobre el link de referencia de cada libro.

Analizar si es necesario aplicar técnicas para limpiar la descripcion de cada libro

In [26]:
# ANALIZAR GUARDAR ESTE DF PARA NO VOLVER A CORRER EL CODIGO, TARDA COMO 3 MINS


for index, link in enumerate(books_df['link']):

  url = f"https://www.gutenberg.org{link}"
  # url= 'https://www.gutenberg.org/ebooks/348'

  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  # Buscar el div con id 'bibrec'
  bio_div = soup.find('div', id='bibrec')

  # Buscar el tr que contiene el th con el texto 'Summary'
  summary_tr = bio_div.find('th', string='Summary')

  # Si encontró el 'th' correcto, obtener el contenido del 'td' siguiente
  if summary_tr:
      summary_text = summary_tr.find_parent('tr').find('td').get_text(strip=True)
  else:
      summary_text = 'Descripción no encontrada'

  books_df.at[index, 'summary'] = summary_text


In [27]:
books_df.sample(20)

,title,link,summary
887,The Golden Bough: A Study of Magic and Religio...,/ebooks/3623,"""The Golden Bough: A Study of Magic and Religi..."
798,The Marching Morons by C. M. Kornbluth (56),/ebooks/51233,"""The Marching Morons"" by C. M. Kornbluth is a ..."
36,A Tale of Two Cities by Charles Dickens (757),/ebooks/98,"""A Tale of Two Cities"" by Charles Dickens is a..."
786,Demian by Hermann Hesse (57),/ebooks/74222,"""Demian"" by Hermann Hesse is a novel written i..."
481,Darkwater: Voices from Within the Veil by W. E...,/ebooks/15210,"""Darkwater: Voices from Within the Veil"" by W...."
943,The Suppression of the African Slave Trade to ...,/ebooks/17700,"""The Suppression of the African Slave Trade to..."
309,"The Ladies' Book of Etiquette, and Manual of P...",/ebooks/35123,"""The Ladies' Book of Etiquette, and Manual of ..."
463,The Memoirs of Sherlock Holmes by Arthur Conan...,/ebooks/834,"""The Memoirs of Sherlock Holmes"" by Arthur Con..."
742,The Gilded Age: A Tale of Today by Mark Twain ...,/ebooks/3178,"""The Gilded Age: A Tale of Today"" by Mark Twai..."
319,Troilus and Criseyde by Geoffrey Chaucer (118),/ebooks/257,"""Troilus and Criseyde"" by Geoffrey Chaucer is ..."


In [28]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1000 non-null   object
 1   link     1000 non-null   object
 2   summary  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [29]:
(books_df['summary'] == 'Descripción no encontrada').sum()

183

In [30]:
df_books_filtered : pd.DataFrame = books_df.copy()
df_books_filtered['title'] = df_books_filtered['title'].str.replace(r'\s*\(\d+\)$', '', regex=True)

In [31]:
df_books_filtered.to_csv('/content/books.csv')

## Modelado
- Se utiliza transformers para clasificar el estado de ánimo de la persona. Este puede ser: "Alegre", "Melancólico" o "Ni fu ni fa".

In [40]:
# Cargamos el tokenizador y el modelo.
model_name: str = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer: BertTokenizer = BertTokenizer.from_pretrained(model_name)
model: BertForSequenceClassification = BertForSequenceClassification.from_pretrained(model_name)


# Creación de pipeline.
nlp: pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [51]:
def get_sentiment(text: str) -> str:
    """
    Retorna el sentimiento de un texto, mediante el uso de transformers.
    """
    label: str = nlp(text)[0]['label']
    score: float = nlp(text)[0]['score']

    if label in ['5 stars', '4 stars']:
        print(score)
        return 'Alegre'
    elif label == '3 stars':
        print(score)
        return 'Ni fu ni fa'
    elif label in ['2 stars', '1 star']:
        print(score)
        return 'Melancólico'
    else:
        return 'Ni fu ni fa'


In [52]:
test_list: list[str] = [
    # Frases alegres
    "No puedo esperar para sentir la brisa del mar y disfrutar del sol en la playa.",
    "Tengo muchas ganas de jugar al voleibol en la arena con mis amigos.",
    "Estoy emocionado por las cenas al aire libre y las caminatas por la playa.",

    # Frases neutras
    "Voy a la playa, pero espero que el clima no arruine mis planes.",
    "Me gustaría leer un buen libro si no puedo salir por la lluvia.",
    "Aún estoy organizando lo que haré durante mis vacaciones.",

    # Frases melancólicas
    "No estoy seguro de que la lluvia me deje disfrutar de mis vacaciones.",
    "Me siento un poco decepcionado por la posibilidad de mal tiempo.",
    "Ojalá pudiera cambiar el clima para disfrutar más de mis días libres.",

    # Frases de frustración
    "No sé qué hacer si llueve todos los días de mis vacaciones.",
    "Quiero disfrutar, pero la lluvia arruina todo.",
    "Me molesta pensar que no podré hacer nada divertido en la playa.",

    # Frases optimistas
    "Si llueve, siempre hay buenas películas que puedo ver.",
    "Aprovecharé los días de lluvia para leer más libros que tengo pendientes.",
    "Estoy listo para adaptarme; siempre puedo jugar a algo divertido en casa."
]

for text in test_list:
  print(f'{text} -> {get_sentiment(text)}')

0.32372063398361206
No puedo esperar para sentir la brisa del mar y disfrutar del sol en la playa. -> Alegre
0.575255274772644
Tengo muchas ganas de jugar al voleibol en la arena con mis amigos. -> Alegre
0.5418862700462341
Estoy emocionado por las cenas al aire libre y las caminatas por la playa. -> Alegre
0.5084640383720398
Voy a la playa, pero espero que el clima no arruine mis planes. -> Ni fu ni fa
0.39998897910118103
Me gustaría leer un buen libro si no puedo salir por la lluvia. -> Alegre
0.5287613272666931
Aún estoy organizando lo que haré durante mis vacaciones. -> Alegre
0.39217862486839294
No estoy seguro de que la lluvia me deje disfrutar de mis vacaciones. -> Ni fu ni fa
0.4797669053077698
Me siento un poco decepcionado por la posibilidad de mal tiempo. -> Melancólico
0.32460734248161316
Ojalá pudiera cambiar el clima para disfrutar más de mis días libres. -> Ni fu ni fa
0.38441574573516846
No sé qué hacer si llueve todos los días de mis vacaciones. -> Alegre
0.40683263540